In [12]:
import pandas as pd

# Battery Specifications (Tesla Powerwall 3 x1 )
BATTERY_CAPACITY_KWH = 13.5
MAX_CHARGE_RATE_KW = 5  # Continuous power rating

# Battery Specifications (Tesla Powerwall 3 x3 )
# BATTERY_CAPACITY_KWH = 13.5*3
# MAX_CHARGE_RATE_KW = 8  # Continuous power rating

MAX_DISCHARGE_RATE_KW = 11  # Continuous power rating
ROUND_TRIP_EFFICIENCY = 0.89  # Estimated round-trip efficiency


class Battery:
    """Represents the battery with its state and actions."""

    def __init__(self, capacity_kwh, max_charge_rate_kw, max_discharge_rate_kw,
                 round_trip_efficiency):
        self.capacity_kwh = capacity_kwh
        self.max_charge_rate_kw = max_charge_rate_kw
        self.max_discharge_rate_kw = max_discharge_rate_kw
        self.round_trip_efficiency = round_trip_efficiency
        self.charge_kwh = 0.0

    def charge(self, amount_kwh):
        """Charges the battery, respecting capacity and rate limits."""
        actual_charge_kwh = min(
            amount_kwh, self.capacity_kwh - self.charge_kwh)
        self.charge_kwh += actual_charge_kwh * self.round_trip_efficiency
        return actual_charge_kwh

    def discharge(self, amount_kwh):
        """Discharges the battery, respecting capacity and rate limits."""
        actual_discharge_kwh = min(amount_kwh, self.charge_kwh)
        self.charge_kwh -= actual_discharge_kwh
        return actual_discharge_kwh


def simulate_battery(data_filepath):
    """
    Simulates a home battery system with variable import/export rates
    and daily price updates at 4 PM, considering the full 24-hour
    price information and battery constraints for decision-making.

    Args:
        data_filepath: Path to the CSV file containing the time-series data.

    Returns:
        A pandas DataFrame with simulation results and the total savings/earnings.
    """

    # Load data
    df = pd.read_csv(data_filepath, parse_dates=[
        'interval_start', 'interval_end'])
    # --- Sort data by time ---
    df.sort_values(by='interval_start', inplace=True)
    df.reset_index(drop=True, inplace=True)  # Reset index after sorting

    # Initialize battery
    battery = Battery(BATTERY_CAPACITY_KWH, MAX_CHARGE_RATE_KW,
                      MAX_DISCHARGE_RATE_KW, ROUND_TRIP_EFFICIENCY)
    total_cost = 0.0
    total_earnings = 0.0

    # Store simulation results
    results = []

    # Function to get the next day's prices
    def get_next_day_prices(current_date):
        next_day_start = current_date.replace(
            hour=16, minute=0) + pd.Timedelta(days=1)
        next_day_end = next_day_start + pd.Timedelta(days=1)
        next_day_prices = df[(df['interval_start'] >= next_day_start) & (df['interval_start'] < next_day_end)][[
            'interval_start', 'buy_rate_inc_vat', 'sell_rate_inc_vat']].set_index('interval_start')
        return next_day_prices

    # Iterate through the time-series data
    for index, row in df.iterrows():
        consumption_kwh = row['consumption']
        import_rate = row['buy_rate_inc_vat']
        export_rate = row['sell_rate_inc_vat'] if 'sell_rate_inc_vat' in row else 0
        current_time = row['interval_start']

        # --- Battery State Machine Logic ---
        initial_charge = battery.charge_kwh

        # Get next day's prices if it's 4 PM
        if current_time.hour == 16 and current_time.minute == 0:
            next_day_prices = get_next_day_prices(current_time)

        # Use next day's prices if available
        if 'next_day_prices' in locals():
            # Sort future prices by profit (descending order)
            future_prices_sorted = next_day_prices.copy()
            future_prices_sorted['profit_charge'] = future_prices_sorted['sell_rate_inc_vat'] - import_rate
            future_prices_sorted['profit_discharge'] = future_prices_sorted['buy_rate_inc_vat'] - export_rate
            future_prices_sorted.sort_values(
                by=['profit_charge', 'profit_discharge'], ascending=[False, False], inplace=True)

            # Charge/Discharge based on sorted profit and battery constraints
            remaining_charge_capacity = BATTERY_CAPACITY_KWH - battery.charge_kwh
            remaining_discharge_capacity = battery.charge_kwh

            for future_time, future_row in future_prices_sorted.iterrows():
                if current_time == future_time:
                    future_import_rate = future_row['buy_rate_inc_vat']
                    future_export_rate = future_row['sell_rate_inc_vat']

                    # Charge if profitable and capacity available
                    if future_export_rate > import_rate and remaining_charge_capacity > 0:
                        charge_amount_kwh = battery.charge(
                            min(MAX_CHARGE_RATE_KW * 0.5, remaining_charge_capacity))
                        remaining_charge_capacity -= charge_amount_kwh
                        total_cost -= charge_amount_kwh * import_rate
                        total_earnings += charge_amount_kwh * future_export_rate

                    # Discharge if profitable and capacity available
                    if future_import_rate > export_rate and remaining_discharge_capacity > 0 and consumption_kwh > 0:
                        discharge_amount_kwh = battery.discharge(
                            min(MAX_DISCHARGE_RATE_KW * 0.5, remaining_discharge_capacity, consumption_kwh))
                        remaining_discharge_capacity -= discharge_amount_kwh
                        consumption_kwh -= discharge_amount_kwh
                        total_cost -= discharge_amount_kwh * import_rate
                        total_earnings += discharge_amount_kwh * export_rate

        # 3. Buy remaining energy from the grid
        if consumption_kwh > 0:
            total_cost += consumption_kwh * import_rate

        # Store results for this interval
        results.append({
            'interval_start': row['interval_start'],
            'interval_end': row['interval_end'],
            'initial_battery_charge_kwh': initial_charge,
            'consumption_kwh': row['consumption'],
            'import_rate': import_rate,
            'export_rate': export_rate,
            'charge_amount_kwh': charge_amount_kwh if 'charge_amount_kwh' in locals() else 0,
            'discharge_amount_kwh': discharge_amount_kwh if 'discharge_amount_kwh' in locals() else 0,
            'final_battery_charge_kwh': battery.charge_kwh,
            'cost_this_interval': (consumption_kwh * import_rate) - (discharge_amount_kwh * import_rate if 'discharge_amount_kwh' in locals() else 0) - (charge_amount_kwh * import_rate if 'charge_amount_kwh' in locals() else 0),
            'earnings_this_interval': (charge_amount_kwh * future_export_rate if 'charge_amount_kwh' in locals() else 0) + (discharge_amount_kwh * export_rate if 'discharge_amount_kwh' in locals() else 0)
        })

    # Create a DataFrame from the results
    results_df = pd.DataFrame(results)

    # Calculate total savings and earnings from the entire simulation
    net_savings_earnings = total_earnings + total_cost  # Cost is negative savings

    return results_df, net_savings_earnings


# --- Example Usage ---
data_filepath = 'output.csv'  # Replace with your data file
simulation_results, total_savings_earnings = simulate_battery(data_filepath)

e:\Eshwar\Projects\Home Battery\.venv\Lib\site-packages\pandas\core\indexes\base.py:7588: FutureWarning: Dtype inference on a pandas object (Series, Index, ExtensionArray) is deprecated. The Index constructor will keep the original dtype in the future. Call `infer_objects` on the result to get the old behavior.
  return Index(sequences[0], name=names)
e:\Eshwar\Projects\Home Battery\.venv\Lib\site-packages\pandas\core\indexes\base.py:7588: FutureWarning: Dtype inference on a pandas object (Series, Index, ExtensionArray) is deprecated. The Index constructor will keep the original dtype in the future. Call `infer_objects` on the result to get the old behavior.
  return Index(sequences[0], name=names)
e:\Eshwar\Projects\Home Battery\.venv\Lib\site-packages\pandas\core\indexes\base.py:7588: FutureWarning: Dtype inference on a pandas object (Series, Index, ExtensionArray) is deprecated. The Index constructor will keep the original dtype in the future. Call `infer_objects` on the result to g

In [13]:

# Print the results
# print(simulation_results)
days = (simulation_results['interval_end'].max() -
        simulation_results['interval_start'].min()).days
print(
    f"\nTotal Savings/Earnings from Battery per year: £{total_savings_earnings*365/days/100:.2f}")
simulation_results.to_csv('simulation_results.csv', index=False)
# You can further analyze the simulation_results DataFrame to
# understand battery behavior, charge/discharge patterns, etc.


Total Savings/Earnings from Battery per year: £1442.30
